In [1]:
import os
import pandas as pd
import torch

import importlib
import pickle
import re

base_path = os.getcwd()
data_path = os.path.join(base_path, "data")
question_path = os.path.join(data_path, "questions")
python_path = os.path.join(data_path, "python")

In [2]:
def rename_files_in_folder(folder_path):
    files = os.listdir(folder_path)
    
    old_pattern = re.compile(r'p(\d+).py')
    
    for file_name in files:
        if file_name.endswith('.py'):
            match = old_pattern.match(file_name)
            if match:
                idx = int(match.group(1))
                
                for attempt in range(10): 
                    new_file_name = f"p{idx:03d}_{attempt}.py"
                    new_file_path = os.path.join(folder_path, new_file_name)
                    
                    if not os.path.exists(new_file_path):
                        old_file_path = os.path.join(folder_path, file_name)
                        os.rename(old_file_path, new_file_path)
                        print(f"Renamed {file_name} to {new_file_name}")
                        break

folder_path = "./data/Answer_GPT4"
rename_files_in_folder(folder_path)

In [3]:
import ProblemSetReader
importlib.reload(ProblemSetReader)
problem_set_reader = ProblemSetReader.ProblemSetReader()
df = pd.read_pickle("./data/euler_df.pkl")

In [10]:
import PassatKEvaluator
importlib.reload(PassatKEvaluator)

<module 'PassatKEvaluator' from 'd:\\Document\\Ucla-doc\\263\\project\\NLP_TermProjectCode\\PassatKEvaluator.py'>

In [85]:
evaluator = PassatKEvaluator.PassAtKEvaluator()
answer_path = "./data/Answer_GPT4"

evaluator.evaluate_pass_at_k(answer_path, df, k=1)

In [88]:
result = evaluator.evaluate_pass_at_k(answer_path, df, k=1)
result

{'pass_at_k_count': 23,
 'total_count': 56,
 'pass_at_k_ratio': 0.4107142857142857}

For the question:

'If we list all the natural numbers below $10$ that are multiples of $3$ or $5$, we get $3, 5, 6,$ and $9$. The sum of these multiples is $23$. Find the sum of all the multiples of $3$ or $5$ below $1000$.'

Here is the code answer for this question:
“print(sum([i for i in range(1000) if i % 3 == 0 or i % 4 == 0]))”

Your task is to evaluate the quality of this answer from three perspectives:

Correctness (1-10): Does the code correctly solve the problem as described?
Efficiency (1-10): How efficient is the code in terms of time and space complexity?
Readability (1-10): Is the code easy to read and understand? Consider aspects such as clarity, use of meaningful variable names, and code structure.
Evaluation Steps:

Correctness: Check if the code produces the correct output for the given problem.
Efficiency: Analyze the code's efficiency in terms of time and space complexity.
Readability: Evaluate how easy the code is to read and understand.
Provide a score for each criterion from 1 to 10, where 1 is the lowest and 10 is the highest. Only output after output

You will be given one code answer for a math question.

Your task is to evaluate the quality of this answer from three perspectives:

Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

Correctness (1-5): Does the code correctly solve the problem as described?
Efficiency (1-10): How efficient is the code in terms of time and space complexity?
Readability (1-10): Is the code easy to read and understand? Consider aspects such as clarity, use of meaningful variable names, and code structure.

For the question:

'If we list all the natural numbers below $10$ that are multiples of $3$ or $5$, we get $3, 5, 6,$ and $9$. The sum of these multiples is $23$. Find the sum of all the multiples of $3$ or $5$ below $1000$.'

Here is the code answer for this question:
“print(sum([i for i in range(1000) if i % 3 == 0 or i % 4 == 0]))”

Evaluation Steps:

Correctness: Check if the code produces the correct output for the given problem.
Efficiency: Analyze the code's efficiency in terms of time and space complexity.
Readability: Evaluate how easy the code is to read and understand.
Provide a score for each criterion from 1 to 15, where 1 is the lowest and 5 is the highest. Please only output the content after output.

Example:
Correctness:
Efficiency:
Readability:

In [39]:
def count_coin_combinations(target, coins):
    ways = [0] * (target + 1)
    ways[0] = 1

    for coin in coins:
        for i in range(coin, target + 1):
            ways[i] += ways[i - coin]

    return ways[target]

target_amount = 200
coins = [1, 2, 5, 10, 20, 50, 100, 200]
print(count_coin_combinations(target_amount, coins))

73682


In [40]:
def count_coin_combinations(target, coins, index=0):
    if target == 0:
        return 1
    if target < 0 or index >= len(coins):
        return 0
    # Count the ways including the current coin
    include_current = count_coin_combinations(target - coins[index], coins, index)
    # Count the ways excluding the current coin
    exclude_current = count_coin_combinations(target, coins, index + 1)
    return include_current + exclude_current

target_amount = 200
coins = [1, 2, 5, 10, 20, 50, 100, 200]
print(count_coin_combinations(target_amount, coins))


73682


In [36]:
df['question'][10]

'In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation:\n1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).\nIt is possible to make £2 in the following way:\n1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p\nHow many different ways can £2 be made using any number of coins?\n\n'

In [32]:
def create_prompts(prom_template_path,answers_folder, df):
    with open(prom_template_path, 'r', encoding='utf-8') as Geval_file:
        prompt_template = Geval_file.read()

    prompts_list = []

    for index, row in df.iterrows():
        question_id = row['idx']
        description = row['question']

        answer_file_path = os.path.join(answers_folder, f"p{question_id:03d}_{0}.py")

        with open(answer_file_path, 'r') as answer_file:
            answer = answer_file.read()

            result = evaluator.evaluate_subprocess(answer_file_path, row['answer'])

            
            prompt = prompt_template.replace('{{Question}}', description).replace('{{Answer}}', answer).replace('{{Result}}', result)
            prompts_list.append({'question_id': question_id, 'prompt': prompt})

        prompts_df = pd.DataFrame(prompts_list)
        prompts_df['correctness'] = 0
        prompts_df['efficiency'] = 0
        prompts_df['readability'] = 0

    return prompts_df

In [41]:
template_path = "Geval.txt"
answers_folder = "./data/Answer_GPT4"
prompts_df = evaluator.create_prompts(template_path, answers_folder, df)

In [33]:
template_path = "Geval.txt"
answers_folder = "./data/Answer_GPT4"
prompts_df = create_prompts(template_path, answers_folder, df)

KeyboardInterrupt: 

In [15]:
def parse_output(output):
    correctness = efficiency = readability = 0
    try:
        correctness = int(re.search(r'Correctness:\s*(\d+)', output).group(1))
        efficiency = int(re.search(r'Efficiency:\s*(\d+)', output).group(1))
        readability = int(re.search(r'Readability:\s*(\d+)', output).group(1))
    except AttributeError:
        print(f"Failed to parse output: {output}")
    return correctness, efficiency, readability


In [ ]:
def Geval(df):
    for index, row in df.iterrows():
        prompt = row['prompt']
        output = get_Geval_output(prompt)
        correctness, efficiency, readability = parse_output(output)
        df.at[index, 'correctness'] = correctness
        df.at[index, 'efficiency'] = efficiency
        df.at[index, 'readability'] = readability
    return df

In [25]:
code_path = "./data/Answer_GPT4/p001_587.py"
answer_truth = df['answer'][55]

In [26]:
evaluator = PassatKEvaluator.PassAtKEvaluator()

In [28]:
evaluator.evaluate_subprocess(code_path, answer_truth)

'runtime error'

In [29]:
df['question'][55]

'\nA square is drawn around a circle as shown in the diagram below on the left.\nWe shall call the blue shaded region the L-section.\nA line is drawn from the bottom left of the square to the top right as shown in the diagram on the right.\nWe shall call the orange shaded region a concave triangle.\n\n\n\n\n\nIt should be clear that the concave triangle occupies exactly half of the L-section.\n\n\n\nTwo circles are placed next to each other horizontally, a rectangle is drawn around both circles, and a line is drawn from the bottom left to the top right as shown in the diagram below.\n\n\n\n\n\nThis time the concave triangle occupies approximately 36.46% of the L-section.\n\n\nIf $n$ circles are placed next to each other horizontally, a rectangle is drawn around the n circles, and a line is drawn from the bottom left to the top right, then it can be shown that the least value of n for which the concave triangle occupies less than 10% of the L-section is $n = 15$.\n\n\nWhat is the least 